# Lab 4

In [ ]:
# %pip install geemap

In [ ]:
# Import packages
import ee
import geemap

In [ ]:
geemap.ee_initialize()

In [ ]:
# Check initialize
Map = geemap.Map()
#Map

# Question 1

Write a program to find out how many counties are named Knox in the US.

In [ ]:
Map1 = geemap.Map()

# Load borders and filter by name
borders = ee.FeatureCollection("TIGER/2018/Counties")
knox = borders.filter(ee.Filter.eq('NAME', 'Knox'))

# Convert to df
geemap.ee_to_df(knox)

,ALAND,AWATER,CBSAFP,CLASSFP,COUNTYFP,COUNTYNS,CSAFP,FUNCSTAT,GEOID,INTPTLAT,INTPTLON,LSAD,METDIVFP,MTFCC,NAME,NAMELSAD,STATEFP
0,1855569287,8837382,23660,H1,095,00424249,,A,17095,+40.9309446,-090.2137921,06,,G4020,Knox,Knox County,17
1,1336511196,20758105,47180,H1,083,00450366,,A,18083,+38.6883910,-087.4203544,06,,G4020,Knox,Knox County,18
2,1000523138,3743713,30940,H1,121,00516907,,A,21121,+36.8874755,-083.8556297,06,,G4020,Knox,Knox County,21
3,945684684,2017116243,,H1,013,00581292,,A,23013,+44.0420446,-069.0385145,06,,G4020,Knox,Knox County,23
4,1305376626,7307094,,H1,103,00758506,,A,29103,+40.1368559,-092.1468345,06,,G4020,Knox,Knox County,29
5,2870856403,81009708,,H1,107,00835875,,A,31107,+42.6344045,-097.8913492,06,,G4020,Knox,Knox County,31
6,1361052752,10720134,34540,H1,083,01074054,198,A,39083,+40.4036198,-082.4223929,06,,G4020,Knox,Knox County,39
7,1316572232,45558452,28940,H1,093,01639761,314,A,47093,+35.9927265,-083.9377209,06,,G4020,Knox,Knox County,47
8,2203109026,12577260,,H1,275,01383923,,A,48275,+33.6118897,-099.7303563,06,,G4020,Knox,Knox County,48


![](https://i.imgur.com/lce1rDu.jpeg)

# Question 2

Display Knox county of Tennesse with outline only (no fill color) on the map. (Hint: The STATEFP of Tennessee is 47)

In [ ]:
Map1 = geemap.Map()

# Load borders and filter by name
borders = ee.FeatureCollection("TIGER/2018/Counties")
knox = borders.filter(ee.Filter.eq('NAME', 'Knox'))
knox_tn = knox.filter(ee.Filter.eq('STATEFP', '47'))

# set style border
style = {'color': 'ff0000ff', 'width': 2, 'lineType': 'solid', 'fillColor': 'FF000000'}

# Add text to the map
text = "Created by Pablo Miranda"
Map1.add_text(text, fontsize=20, position='bottomright')

# Add Layers
Map1.centerObject(knox_tn, 10)
Map1.addLayer(knox_tn.style(**style), {}, 'knox_tn')

# Display the map
Map1

Map(center=[35.993196068178555, -83.93723059189078], controls=(WidgetControl(options=['position', 'transparent…

![](https://i.imgur.com/sFGZu7M.jpeg)

# Question 3

Use Landsat-9 data to create a cloud-free imagery for Knox County, TN. Display the imagery on the map with a proper band combination.



In [ ]:
Map2 = geemap.Map()

# Load borders and filter by name
borders = ee.FeatureCollection("TIGER/2018/Counties")
knox = borders.filter(ee.Filter.eq('NAME', 'Knox'))
knox_tn = knox.filter(ee.Filter.eq('STATEFP', '47'))

# Load Landsat dataset
collection = (
    ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")
    .filterDate('2021-01-01', '2023-01-01') # two years of data
    .filterBounds(knox_tn) # intersecting ROI
    .filter(ee.Filter.lt('CLOUD_COVER', 5))
)

# Create cloud-free imagery
image = collection.median()

# Clip it
cloudfree = image.clip(knox_tn)

vis = {
    'min': 0.0,
    'max': 45000,
    'bands': ['SR_B6', 'SR_B5', 'SR_B2'], # Agriculture combination
}


# set style border
style = {'color': '00ff00ff', 'width': 2, 'lineType': 'solid', 'fillColor': 'FF000000'}

# Add text to the map
text = "Created by Pablo Miranda"

# Add Layers
Map2.add_text(text, fontsize=20, position='bottomright')
Map2.addLayer(knox_tn.style(**style), {}, 'Knox')
Map2.centerObject(knox_tn, 10)
Map2.addLayer(cloudfree, vis, 'Landsat-9')
Map2


Map(center=[35.993196068178555, -83.93723059189078], controls=(WidgetControl(options=['position', 'transparent…

![](https://i.imgur.com/0G8TJDj.jpeg)

# Question 4

Use Sentinel-2 data to create a cloud-free imagery for Knox County, TN. Display the imagery on the map with a proper band combination.


In [ ]:
Map3 = geemap.Map()

# Load borders and filter by name
borders = ee.FeatureCollection("TIGER/2018/Counties")
knox = borders.filter(ee.Filter.eq('NAME', 'Knox'))
knox_tn = knox.filter(ee.Filter.eq('STATEFP', '47'))

# Load Sentinel dataset
collection = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterDate('2021-01-01', '2023-01-01') # two years of data
    .filterBounds(knox_tn) # intersecting ROI
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 5))
)

# Create cloud-free imagery
image = collection.median()

# Clip it
cloudfree = image.clip(knox_tn)

vis = {
    'min': -200,
    'max': 3000,
    'bands':['B8', 'B4', 'B3'], # Color Infrared
}

# set style border
style = {'color': '00ff00ff', 'width': 2, 'lineType': 'solid', 'fillColor': 'FF000000'}

# Add text to the map
text = "Created by Pablo Miranda"

# Add Layers
Map3.add_text(text, fontsize=20, position='bottomright')
Map3.addLayer(knox_tn.style(**style), {}, 'Knox')
Map3.centerObject(knox_tn, 10)
Map3.addLayer(cloudfree, vis, 'Sentinel-2')
Map3

Map(center=[35.993196068178555, -83.93723059189078], controls=(WidgetControl(options=['position', 'transparent…

![](https://i.imgur.com/AqbuWve.jpeg)


# Question 5
Use NAIP imagery to create a cloud-free imagery for Knox County, TN. Display the imagery on the map with a proper band combination.

In [ ]:
Map4 = geemap.Map()

# Load borders and filter by name
borders = ee.FeatureCollection("TIGER/2018/Counties")
knox = borders.filter(ee.Filter.eq('NAME', 'Knox'))
knox_tn = knox.filter(ee.Filter.eq('STATEFP', '47'))

# Load USDA dataset
collection = (
    ee.ImageCollection('USDA/NAIP/DOQQ')
    .filter(ee.Filter.date('2017-01-01', '2018-12-31')) # 2017 year cicle
    .filterBounds(knox_tn)
    )

# Create a mosaic
image = collection.mosaic()

# Clip
trueColor = image.clipToCollection(knox_tn)

trueColor = trueColor.select(['R', 'G', 'B'])
trueColorVis = {
  min: 0,
  max: 255,
};

# set style border
style = {'color': '00ff00ff', 'width': 2, 'lineType': 'solid', 'fillColor': 'FF000000'}

# Add text to the map
text = "Created by Pablo Miranda"

# Add Layers
Map4.add_text(text, fontsize=20, position='bottomright')
Map4.addLayer(knox_tn.style(**style), {}, 'Knox')
Map4.centerObject(knox_tn, 10)
Map4.addLayer(trueColor, trueColorVis, 'True Color');
Map4

Map(center=[35.993196068178555, -83.93723059189078], controls=(WidgetControl(options=['position', 'transparent…

![](https://i.imgur.com/w8vbcMi.jpeg)

